# IIC-3800 Tópicos en CC - NLP UC

! pip install datasets transformers

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import transformers

print(transformers.__version__)

4.25.1


In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

Fine-tuning a DistilGPT2 con Wikipedia

In [4]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Found cached dataset wikitext (C:/Users/marce/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets["train"])

,text
0,"L.A.M.B. has participated in the Spring / Summer 2006 , 2007 , and 2008 New York Fashion Weeks . Stefani described her first line , which debuted on September 16 , 2005 , as "" a little Sound of Music , some Orange County chola girl , some Rasta , and a bit of The Great Gatsby . "" The highlights of the show were purple cars bouncing using hydraulics while Stefani 's song "" Wind It Up "" made its debut as the models walked the runway . \n"
1,"In January 2011 , one year after the quake , Oxfam published a report on the status of the recovery . According to the report , relief and recovery were at a standstill due to government inaction and indecision on the part of the donor countries . The report stated , "" One year on , only five percent of the rubble has been cleared and only 15 percent of the required basic and temporary houses have been built . House building on a large scale cannot be started before the enormous amount of rubble is cleared . The government and donors must prioritize this most basic step toward helping people return home "" . Robert Fox , executive director of Oxfam Canada , said "" The dysfunction has been aided unabated by the way the international community has organized itself , where pledges have been made and they haven 't followed through [ and ] where they come to the table with their own agendas and own priorities . Most donors provided funds for transitional housing but very little money for clearing rubble or repairing houses "" . Fox said that in many instances rubble removal "" means it was [ moved ] off someone 's property onto the road in front of the property "" . According to a UNICEF report , "" Still today more than one million people remain displaced , living in crowded camps where livelihoods , shelter and services are still hardly sufficient for children to stay healthy "" . Amnesty International reported that armed men were preying with impunity on girls and women in displacement camps , worsening the trauma of having lost homes , livelihoods and loved ones . \n"
2,
3,
4,
5,
6,"Their opponents in the second round were Hungarian champions Honvéd . A goal from Santillana ensured Real won the first leg 1 – 0 in Spain . Two goals from Laurie Cunningham and Francisco García Hernández secured a 2 – 0 victory in the second leg at Honvéd 's home ground the Bozsik Stadion , thus , winning the tie 3 – 0 on aggregate . \n"
7,
8,= = Relationships with humans = = \n
9,


In [7]:
model_checkpoint = "distilgpt2"

In [8]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(examples["text"])

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [10]:
tokenizer_wrapper = TokenizerWrapper(tokenizer)

In [11]:
tokenized_datasets = datasets.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-30009711dc059d99_*_of_00004.arrow
Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-6bbf9169baf7c5fb_*_of_00004.arrow
Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-207565d47927079f_*_of_00004.arrow


In [12]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
def group_texts(examples):
    block_size = 128
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-8da755f5ffa8a975_*_of_00004.arrow
Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-fb04cb0963cad339_*_of_00004.arrow
Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-b98f68ea47ed009c_*_of_00004.arrow


In [15]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

In [16]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [17]:
from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [19]:
trainer.train()

C:\Users\marce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18666
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7002
  Number of trainable parameters = 81912576


Epoch,Training Loss,Validation Loss
1,3.759600,3.665788
2,3.651900,3.647420
3,3.601400,3.642290


Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-500
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-500\config.json
Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-500\pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-1000
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-1000\config.json
Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-1500
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-1500\config.json
Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-2000
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-2000\config.json
Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-2000\pytorch_model.bin
***** Running Evaluation **

TrainOutput(global_step=7002, training_loss=3.6941190673024, metrics={'train_runtime': 465.2762, 'train_samples_per_second': 120.354, 'train_steps_per_second': 15.049, 'total_flos': 1829011929956352.0, 'train_loss': 3.6941190673024, 'epoch': 3.0})

In [20]:
trainer.save_model("distilgpt2-finetuned.h5")

Saving model checkpoint to distilgpt2-finetuned.h5
Configuration saved in distilgpt2-finetuned.h5\config.json
Model weights saved in distilgpt2-finetuned.h5\pytorch_model.bin


In [21]:
from transformers import DistilBertConfig, DistilBertModel

model = DistilBertModel.from_pretrained("distilgpt2-finetuned.h5")

loading configuration file distilgpt2-finetuned.h5\config.json
You are using a model of type gpt2 to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Model config DistilBertConfig {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation": "gelu",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attention_dropout": 0.1,
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dim": 768,
  "dropout": 0.1,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_heads": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_layers": 6,
  "n_positions": 1024,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "reorder_and_upcast_attn":

Masked Language Modeling

In [22]:
model_checkpoint = "distilroberta-base"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\marce/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e1793fd9ef58390b40a80a4c10df32\config.json
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at C:\Users\ma

In [24]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-8da755f5ffa8a975_*_of_00004.arrow
Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-fb04cb0963cad339_*_of_00004.arrow
Loading cached processed dataset at C:\Users\marce\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-b98f68ea47ed009c_*_of_00004.arrow


In [25]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at C:\Users\marce/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e1793fd9ef58390b40a80a4c10df32\config.json
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at C:\Users\marce/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e179

In [26]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [27]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [29]:
trainer.train()

C:\Users\marce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18666
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7002
  Number of trainable parameters = 82170201
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,5.915600,5.779372
2,5.670700,5.579891
3,5.551200,5.498873


Saving model checkpoint to distilroberta-base-finetuned-wikitext2\checkpoint-500
Configuration saved in distilroberta-base-finetuned-wikitext2\checkpoint-500\config.json
Model weights saved in distilroberta-base-finetuned-wikitext2\checkpoint-500\pytorch_model.bin
Saving model checkpoint to distilroberta-base-finetuned-wikitext2\checkpoint-1000
Configuration saved in distilroberta-base-finetuned-wikitext2\checkpoint-1000\config.json
Model weights saved in distilroberta-base-finetuned-wikitext2\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to distilroberta-base-finetuned-wikitext2\checkpoint-1500
Configuration saved in distilroberta-base-finetuned-wikitext2\checkpoint-1500\config.json
Model weights saved in distilroberta-base-finetuned-wikitext2\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to distilroberta-base-finetuned-wikitext2\checkpoint-2000
Configuration saved in distilroberta-base-finetuned-wikitext2\checkpoint-2000\config.json
Model weights saved in dist

TrainOutput(global_step=7002, training_loss=5.8092338602327, metrics={'train_runtime': 429.523, 'train_samples_per_second': 130.373, 'train_steps_per_second': 16.302, 'total_flos': 1856638981384704.0, 'train_loss': 5.8092338602327, 'epoch': 3.0})

In [30]:
trainer.save_model("distilroberta-finetuned.h5")

Saving model checkpoint to distilroberta-finetuned.h5
Configuration saved in distilroberta-finetuned.h5\config.json
Model weights saved in distilroberta-finetuned.h5\pytorch_model.bin


In [31]:
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained("distilroberta-finetuned.h5")

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

loading file vocab.json from cache at C:\Users\marce/.cache\huggingface\hub\models--roberta-base\snapshots\bc2764f8af2e92b6eb5679868df33e224075ca68\vocab.json
loading file merges.txt from cache at C:\Users\marce/.cache\huggingface\hub\models--roberta-base\snapshots\bc2764f8af2e92b6eb5679868df33e224075ca68\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\marce/.cache\huggingface\hub\models--roberta-base\snapshots\bc2764f8af2e92b6eb5679868df33e224075ca68\config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_

In [32]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0287,  0.1037,  0.0497,  ..., -0.1419, -0.0878, -0.1035],
         [-0.1012,  0.1349,  0.0487,  ..., -0.1946, -0.0503,  0.0087],
         [-0.0154,  0.2017, -0.0121,  ..., -0.5496,  0.0703,  0.0294],
         ...,
         [-0.0099,  0.0509, -0.0503,  ...,  0.2083,  0.2238, -0.1072],
         [-0.0450,  0.1117, -0.0041,  ..., -0.1812, -0.0819, -0.1107],
         [-0.0152,  0.0821,  0.0414,  ..., -0.0479, -0.0595, -0.0274]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.7856e-02, -2.6244e-02,  3.7591e-02, -4.3138e-02, -6.1570e-04,
          4.6531e-01,  1.3777e-01,  1.7208e-01, -4.9658e-01, -1.8586e-02,
          3.8688e-01,  1.5355e-01, -3.2527e-01,  4.8366e-02,  1.6435e-01,
         -3.6283e-01,  3.4307e-01, -2.0699e-01, -8.2975e-02,  7.2161e-02,
         -2.0497e-02,  3.7040e-01, -2.3906e-01,  1.6113e-01,  5.1800e-01,
         -4.1507e-01, -4.0727e-01, -5.0621e-01,  4.3897e-01,  2.930

Using pretrained LLMs (sentence transformers)

In [33]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


loading file vocab.txt from cache at C:\Users\marce/.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\7dbbc90392e2f80f3d3c277d6e90027e55de9125\vocab.txt
loading file tokenizer.json from cache at C:\Users\marce/.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\7dbbc90392e2f80f3d3c277d6e90027e55de9125\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\marce/.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\7dbbc90392e2f80f3d3c277d6e90027e55de9125\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\marce/.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\7dbbc90392e2f80f3d3c277d6e90027e55de9125\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\marce/.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\7db

In [34]:
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0985e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2531e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0638e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6243e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4717e